In [23]:
import os
len(os.listdir("/kaggle/input/preprocessed-leaf/preprocessed_data_test"))
# os.mkdir("models")
# os.listdir("/kaggle/working")

# os.listdir("/kaggle/input/plant_disease/pytorch/default/1")

39

In [12]:
open("/kaggle/working/abc.txt","w").write("dsfsdfs")

7

In [2]:
import torch

torch.cuda.is_available()

torch.cuda.device_count()


torch.cuda.current_device()


torch.cuda.device(0)


torch.cuda.get_device_name(0)



AssertionError: Torch not compiled with CUDA enabled

In [6]:
import os
import torch
# import torchvision
from torchvision import datasets, models, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# Set device to GPU if available
device = torch.device("cuda")
torch.cuda.empty_cache()

# Define paths
train_dir = '/kaggle/input/preprocessed-leaf/preprocessed_data_test'
version = 1
model_path = '/kaggle/input/plant_disease/pytorch/default/1/plant_disease_detection_resnet50_finetuned20.pth'

# val_dir = 'Validation'

# Define image transformations for training and validation
train_transforms = transforms.Compose([
    # transforms.RandomResizedCrop(224),
    transforms.Resize((224, 224)),  # Resizing the image to 224x224
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=480, shuffle=True, num_workers=0)

num_classes = 39  # Change this to the number of classes in your dataset

# Initialize the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers of the base model
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes)
)


# Load the model state from the 10th epoch
model.load_state_dict(torch.load(model_path))

model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

def train(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

# Resume training from the 11th epoch
start_epoch = 1
num_epochs = 200
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Fine-tune by unfreezing some layers of the base model
for param in model.layer4.parameters():
    param.requires_grad = True

# Lower the learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Fine-tune the model
fine_tune_epochs = 200
start_fine_tune_epoch = 1  # If fine-tuning from scratch, else adjust this value accordingly
print("Begingin Training")
for epoch in range(start_fine_tune_epoch, fine_tune_epochs+1):
    print(f"Fine-Tuning Epoch {epoch}/{fine_tune_epochs}")

    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    # val_loss, val_acc = validate(model, val_loader, criterion)

    train_losses.append(train_loss)
    # val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    # val_accuracies.append(val_acc)

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    os.makedirs(f"models/{version}", exist_ok=True)
    torch.save(model.state_dict(), f'models/{version}/plant_disease_detection_resnet50_finetuned_{epoch}.pth')
    torch.save(model,f'models/{version}/plant_disease_detection_resnet50_finetuned_{epoch}_model.pth')
    # print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')

# Final model save
torch.save(model.state_dict(), 'plant_disease_detection_resnet50_finetuned_final.pth')

torch.save(model,"plant_disease_detection_resnet50_finetuned_final_model.pth")

/tmp/ipykernel_30/1939249203.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Begingin Training
Fine-Tuning Epoch 1/200


Train Loss: 0.3163, Train Accuracy: 89.93%
Fine-Tuning Epoch 2/200


Train Loss: 0.2420, Train Accuracy: 92.32%
Fine-Tuning Epoch 3/200


Train Loss: 0.2017, Train Accuracy: 93.62%
Fine-Tuning Epoch 4/200


...

Train Loss: 0.0459, Train Accuracy: 98.79%
Fine-Tuning Epoch 40/200


Train Loss: 0.0466, Train Accuracy: 98.78%
Fine-Tuning Epoch 41/200


Train Loss: 0.0462, Train Accuracy: 98.78%
Fine-Tuning Epoch 42/200


Train Loss: 0.0454, Train Accuracy: 98.80%
Fine-Tuning Epoch 43/200


KeyboardInterrupt: 